In [2]:
import torch
print (torch.__version__)

2.6.0+cu124


In [3]:
!unzip "/content/drive/MyDrive/data.zip"

Streaming output truncated to the last 5000 lines.
  inflating: val/n02102040/n02102040_662.JPEG  
  inflating: __MACOSX/val/n02102040/._n02102040_662.JPEG  
  inflating: val/n02102040/n02102040_2470.JPEG  
  inflating: __MACOSX/val/n02102040/._n02102040_2470.JPEG  
  inflating: val/n02102040/n02102040_7222.JPEG  
  inflating: __MACOSX/val/n02102040/._n02102040_7222.JPEG  
  inflating: val/n02102040/n02102040_5120.JPEG  
  inflating: __MACOSX/val/n02102040/._n02102040_5120.JPEG  
  inflating: val/n02102040/ILSVRC2012_val_00023242.JPEG  
  inflating: __MACOSX/val/n02102040/._ILSVRC2012_val_00023242.JPEG  
  inflating: val/n02102040/n02102040_571.JPEG  
  inflating: __MACOSX/val/n02102040/._n02102040_571.JPEG  
  inflating: val/n02102040/n02102040_6262.JPEG  
  inflating: __MACOSX/val/n02102040/._n02102040_6262.JPEG  
  inflating: val/n02102040/n02102040_5272.JPEG  
  inflating: __MACOSX/val/n02102040/._n02102040_5272.JPEG  
  inflating: val/n02102040/n02102040_1620.JPEG  
  inflating: _

In [4]:

from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as f
from torchvision import datasets, transforms
import os

In [5]:
#setup device agnostic coding
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(f"using: {device}")

using: cuda


In [6]:
# setup training and testing directory
train_dir = "/content/train"
test_dir = "/content/val"

In [7]:
#define Transforms

train_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
#load datasets
from torchvision.datasets import ImageFolder
train_data = ImageFolder(root=train_dir, transform=train_transform)
test_data = ImageFolder(root=test_dir, transform=test_transform)
len(train_data), len(test_data)


(9469, 3925)

In [9]:
bactch_size = 32
train_dataloader = DataLoader(train_data, batch_size=bactch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=bactch_size, shuffle=False)

In [10]:
class_dict = train_data.class_to_idx
print(class_dict)

{'n01440764': 0, 'n02102040': 1, 'n02979186': 2, 'n03000684': 3, 'n03028079': 4, 'n03394916': 5, 'n03417042': 6, 'n03425413': 7, 'n03445777': 8, 'n03888257': 9}


In [11]:
# adding folder to the system
import sys
sys.path.append("/content/drive/MyDrive")

In [12]:
from model import  CustomCNN

In [13]:
network = CustomCNN(num_classes=10).to(device)

In [14]:
!pip install torchinfo

In [15]:
from torchinfo import summary
summary(network, input_size=(32, 3, 128, 128))



Layer (type:depth-idx)                   Output Shape              Param #
CustomCNN                                [32, 10]                  --
├─Conv2d: 1-1                            [32, 32, 128, 128]        896
├─ReLU: 1-2                              [32, 32, 128, 128]        --
├─MaxPool2d: 1-3                         [32, 32, 64, 64]          --
├─Conv2d: 1-4                            [32, 64, 64, 64]          18,496
├─ReLU: 1-5                              [32, 64, 64, 64]          --
├─MaxPool2d: 1-6                         [32, 64, 32, 32]          --
├─Conv2d: 1-7                            [32, 128, 32, 32]         73,856
├─ReLU: 1-8                              [32, 128, 32, 32]         --
├─MaxPool2d: 1-9                         [32, 128, 16, 16]         --
├─Linear: 1-10                           [32, 512]                 16,777,728
├─ReLU: 1-11                             [32, 512]                 --
├─Linear: 1-12                           [32, 10]                  5

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from timeit import default_timer as timer
from trainNN import train
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=network.parameters(), lr=0.001)
start_time = timer()
results = train(
    model = network,
    train_dataloader = train_dataloader,
    test_dataloader = test_dataloader,
    optimizer = optimizer,
    loss_fn = loss_fn,
    epochs = 20,
    device = device
)
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Training:   0%|          | 0/296 [00:00<?, ?it/s]

Testing:   0%|          | 0/123 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.6595 | train_acc: 0.4360 | test_loss: 1.4154 | test_acc: 0.5314
[INFO] New best model saved to best_model.pth with test_acc: 0.5314


Training:   0%|          | 0/296 [00:00<?, ?it/s]

In [ ]:
import torch
import torchvision.models as models
import timm

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"using: {device}")

In [ ]:
new_network = timm. create_model('efficientnet_b0', pretrained=True, num_classes=10)
new_network.to(device)

In [ ]:
from torchvision import datasets, transforms

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224,0.225])
])

In [ ]:
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)
len(train_data),len(test_data)

In [ ]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size,shuffle=False)

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks")
sys.path.append("/content/drive/MyDrive")

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from timeit import default_timer as timer
from trainNN import train

loss_fn = nn.CrossEntropyLoss()
optimizer =torch.optim.Adam(params=new_network.parameters(), lr=0.01)
start_time = timer()

results = train(
    model = new_network,
    train_dataloader = train_loader,
    test_dataloader = test_loader,
    loss_fn = loss_fn,
    optimizer = optimizer,
    epochs = 10,
    device = device
)
end_time = timer()
print(f"[INFO] Total Trainning time: {end_time-start_time:.3f} sec")